In [1]:
import os
import re
from tqdm import tqdm
from socket import gethostname

import numpy as np
import pandas as pd
from scipy import constants

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
HBARC = constants.physical_constants["natural unit of action in eV s"][0] * constants.c * 1.e09 # in MeV fm

In [3]:
from numpwdata.densities.models import Density2N, Density1N
from numpwdata.densities.models import Chiral as ChiralInteraction
from numpwdata.files.models import H5File, DatFile
from numpwd.densities import Density, read_h5, read_1N_density

In [4]:
from espressodb.management.utilities.version import get_db_info
HOSTNAME = gethostname().split(".")[-1]
print(HOSTNAME)
print(os.environ["NUMPWDATA_CONFIG_DIR"])
db_name, db_user = get_db_info()
print(db_name, db_user)

jureca
/p/project/cjikp03/koerber1/src/nuc/project-dm/
project-dm-db.sqlite3 


In [5]:
DENSDIR = os.path.join(os.getcwd(), "densities-new")
DENSITIES = os.listdir(DENSDIR)

H5_DENSITIES = [d for d in DENSITIES if d.endswith(".h5")]
DAT_DENSITIES = [d for d in DENSITIES if d.endswith(".dat")]

In [58]:
PATTERNS = (
    r"compton-dens-(?P<nuc>[0-9A-z]+)",
    r"\-(?P<potential>[a-z0-9\+]+)",
    r"\-(?P<order>[a-z0-9\+]+)",
    r"\-cut=(?P<cut>[0-9]+)",
    r"\-(?P<empot>[a-zA-Z]+)",
    r"\-(?P<cmpi>(?:[a-z0-9]+))",
    r"(\-(?P<combine>combine))?",
    r"(\-Lam=(?P<Lam>(?:[\.0-9]+)))?",
    r"(\-c1=(?P<c1>(?:[\-\.0-9]+)))?",
    r"(\-c3=(?P<c3>(?:[\-\.0-9]+)))?",
    r"(\-c4=(?P<c4>(?:[\-\.0-9]+)))?",
    r"(\-cd=(?P<cd>(?:[\-\.0-9]+)))?",
    r"(\-ce=(?P<ce>(?:[\-\.0-9]+)))?",
    r"(\-cE1=(?P<cE1>(?:[\-\.0-9]+)))?",
    r"(\-Lambdanumeric=(?P<lambda>(?:[0-9\.e\+]+)))?",
    r"(\-tnfcut=(?P<tnfcut>(?:[0-9]+)))?",
    r"\-om=(?P<omega>(?:[0-9\.]+E[\+\-][0-9]+))",
    r"\-th=(?P<theta>(?:[0-9\.E\+]+))",
    r"(\-nx=(?P<nx>(?:[0-9]+)))?",
    r"(\-nphi=(?P<nphi>(?:[0-9]+)))?",
    r"(\-np12\=np34\=(?P<np12_np34>(?:[0-9\+]+)))?",
    r"(\-np3\=(?P<np3>(?:[0-9\+]+)))?",
    r"(\-nq4\=nq=(?P<nq4_nq>(?:[0-9\+]+)))?",
    r"\-j12max=(?P<j12max>(?:[0-9]+))",
    r"\-lmax=(?P<lmax>(?:[0-9]+))",
    r"\-lsummax=(?P<lsummax>(?:[0-9]+))",
    r"\-tau4max=(?P<tau4max>(?:[0-9]+))",
    r"\-rho(1b)?",
)
DTYPES = {
    int: ["j12max", "lmax", "lsummax", "tau4max", "cut"],
    float: ["lambda", "omega", "theta", "c1", "c3", "c4", "ce", "cd", "cE1", "Lam"],
}
PATTERN = re.compile("".join(PATTERNS))

In [59]:
print("Debugging h5 density pattern")
dens = H5_DENSITIES[0]
extension = "h5"
print(dens)

for n, pat in enumerate(PATTERNS):
    pattern = re.compile("".join(PATTERNS[:n]))
    if pattern.search(dens) is None:
        print("".join(PATTERNS[:n]))
        raise KeyError
print("No issues")

print("Debugging dat density pattern")
dens = DAT_DENSITIES[0]
extension = "dat"
print(dens)

for n, pat in enumerate(PATTERNS):
    pattern = re.compile("".join(PATTERNS[:n]))
    if pattern.search(dens) is None:
        print("".join(PATTERNS[:n]))
        raise KeyError
print("No issues")

Debugging h5 density pattern
compton-dens-4he-chsms-nlo-cut=3-pCoul-no3nf-om=1.20E+00-th=1.80E+02-j12max=5-lmax=6-lsummax=10-tau4max=0-rho.h5
No issues
Debugging dat density pattern
compton-dens-4he-chsms-n4lo+-cut=3-pCoul-n2lo-combine-Lam=500.000-c1=-1.230-c3=-4.650-c4=3.280-cd=1.00-ce=-0.88-cE1=-1.00-om=3.70E+02-th=1.80E+02-j12max=5-lmax=6-lsummax=10-tau4max=0-rho1b.dat
No issues


In [60]:
def parse_file_names(directory, extension):
    files = [f for f in os.listdir(directory) if f.endswith(f".{extension}")]
    
    pattern = re.compile("".join(PATTERNS))
    densities = pd.DataFrame([pattern.search(el).groupdict() for el in files])
    for dtype, cols in DTYPES.items():
        for col in cols:
            densities[col] = densities[col].astype(dtype)

    densities["qval"] = densities["omega"] / HBARC * 2
    densities["file"] = [os.path.join(DENSDIR, f) for f in files]
    densities["owner"] = "Nogga"
    return (
        densities.sort_values("omega").reset_index(drop=True)
        if not densities.empty
        else densities
    )

In [61]:
h5_df = parse_file_names(DENSDIR, "h5")
dat_df = parse_file_names(DENSDIR, "dat")

Identify columns with not present entries

In [62]:
h5_df.isna().any() | dat_df.isna().any()

nuc          False
potential    False
order        False
cut          False
empot        False
cmpi         False
combine       True
Lam           True
c1            True
c3            True
c4            True
cd            True
ce            True
cE1           True
lambda        True
tnfcut        True
omega        False
theta        False
nx            True
nphi          True
np12_np34     True
np3           True
nq4_nq        True
j12max       False
lmax         False
lsummax      False
tau4max      False
qval         False
file         False
owner        False
dtype: bool

In [63]:
cutoff_map = dat_df.groupby("cut")["Lam"].mean().to_dict()
cutoff_map

{1: 400.0, 2: 450.0, 3: 500.0, 4: 550.0}

apparently, the N4LO+ interactions are only uniqely qunatified when also including cd

In [64]:
counts_dat = dat_df.groupby(["potential", "order", "cut", "empot", "qval"])["file"].count()
counts_h5 = h5_df.groupby(["potential", "order", "cut", "empot", "qval"])["file"].count()

print("not filtering with cd")
print(counts_dat[counts_dat > 1].append(counts_h5[counts_h5 > 1]).reset_index()["order"].unique())

counts_dat = dat_df.groupby(["potential", "order", "cut", "empot", "qval", "cd"])["file"].count()
counts_h5 = h5_df.groupby(["potential", "order", "cut", "empot", "qval", "cd"])["file"].count()

print("filtering with cd")
print(counts_dat[counts_dat > 1].append(counts_h5[counts_h5 > 1]).reset_index()["order"].unique())

not filtering with cd
['n4lo+']
filtering with cd
[]


In [73]:
def map_interaction_kwargs(info):
    tag = "cd={cd:0.2f}".format(cd=info["cd"])
    name = info["potential"]
    order = info["order"].upper()
    if order == "N4LO+":
        name += "-" + order + "-" + tag
    interaction_id_kwargs = dict(
        name=name,
        order=order,
        regulator=str(cutoff_map[info["cut"]]),
        em_potential=info["empot"] == "pCoul",
        tag=tag # Adding tags to make interactions unique
    )
    misc = {key: info.get(key) 
        for key in ["empot", "cut", "cmpi", "combine", "c1", "c3", "c4", "ce", "cE1", "cd"]
    }
    misc = {key: None if (isinstance(val, float) and np.isnan(val)) else val for key, val in misc.items()}
    interaction_update_kwargs = dict(
        publication=None,  # needs update
        misc=misc,
    )
    return interaction_id_kwargs, interaction_update_kwargs

def update_or_create_interaction(interaction_id_kwargs, interaction_update_kwargs, verbose=False):
    interaction, created = ChiralInteraction.objects.get_or_create(
        **interaction_id_kwargs
    )
    if created and verbose:
        print("created DB entry for", interaction)
    for key, val in interaction_update_kwargs.items():
        setattr(interaction, key, val)
    interaction.save()
    return interaction, created

map_interaction_kwargs(dat_df.iloc[0])

({'name': 'chsms-N4LO+-cd=0.65',
  'order': 'N4LO+',
  'regulator': '400.0',
  'em_potential': True,
  'tag': 'cd=0.65'},
 {'publication': None,
  'misc': {'empot': 'pCoul',
   'cut': 1,
   'cmpi': 'n2lo',
   'combine': 'combine',
   'c1': -1.23,
   'c3': -4.65,
   'c4': 3.28,
   'ce': 0.13,
   'cE1': 1.0,
   'cd': 0.65}})

In [74]:
def map_density_kwargs(interaction, file, info):
    assert info["nuc"] == "4he"
    density_id_kwargs = dict(
        nucleus=info["nuc"],
        n_nuc=4,
        interaction=interaction,
        qval="{0:8.5f}".format(info["qval"]),
        thetaval="{0:5.2f}".format(info["theta"]),
        file=file,
    )
    if isinstance(file, H5File):
        dens = read_h5(file.path)
        momentum_info={**dens.mesh_info, **dens.current_info}
    elif isinstance(file, DatFile):
        dens = read_1N_density(file.path)
        momentum_info={key: dens["om_theta"][n] for n, key in enumerate(["omega", "thetaval", "qval", "thetaqval"])}
    else:
        raise TypeError(file)

    density_update_kwargs = dict(
        momentum_info=momentum_info,
        channel_info={
            key: info.get(key) for key in ["j12max", "lmax", "lsummax", "tau4max"]
        },
        mesh_info={
            key: info.get(key) for key in ["nx", "nphi", "np3", "nq4_nq", "np12_np34"]
        },
    )
    return density_id_kwargs, density_update_kwargs

# file = H5File.objects.first()
#interaction = ChiralInteraction.objects.first()

#map_density_kwargs(interaction, file, dat_df.iloc[0])

In [75]:
def update_or_create_density(info, density_kind=Density2N, verbose=False):
    interaction_id_kwargs, interaction_update_kwargs = map_interaction_kwargs(info)
    try:
        interaction, created = update_or_create_interaction(interaction_id_kwargs, interaction_update_kwargs, verbose=verbose)
    except Exception as e:
        print("update_or_create_interaction failed for:")
        print(interaction_id_kwargs)
        print(interaction_update_kwargs)
        raise(e)

    file_kwargs = {"path": info["file"], "hostname": HOSTNAME}
    if info["file"].endswith(".h5"):
        FileClass = H5File
    elif info["file"].endswith(".dat"):
        FileClass = DatFile
    else:
        raise TypeError("Do no know how to parse: " + info["file"])
    file, created = FileClass.objects.get_or_create(**file_kwargs)
    if created and verbose:
        print("created DB entry for", file)

    density_id_kwargs, density_update_kwargs = map_density_kwargs(interaction, file, info)

    if (density_kind is Density2N and not isinstance(file, H5File)) or (density_kind is Density1N and not isinstance(file, DatFile)):
        raise TypeError(f"Unexpected combination for file and denisty kind: {file}, {density_kind}")

    density, created = density_kind.objects.get_or_create(**density_id_kwargs)
    if created and verbose:
        print("created DB entry for", density)

    for key, val in density_update_kwargs.items():
        setattr(density, key, val)
    density.save()

    return density, created

In [76]:
for idx, info in tqdm(list(dat_df.iterrows())):
    update_or_create_density(info, density_kind=Density1N, verbose=False)

100%|██████████| 1920/1920 [02:26<00:00, 13.13it/s]


In [77]:
for idx, info in tqdm(list(h5_df.iterrows())):
    update_or_create_density(info, density_kind=Density2N, verbose=False)

100%|██████████| 1920/1920 [37:14<00:00,  1.16s/it]
